In [1]:
import pandas as pd 

In [10]:
df = pd.read_csv('results.csv')

In [11]:
df.columns


Index(['n', 'n_threads', 'chunk_size', 'time'], dtype='object')

In [19]:
import pandas as pd
import numpy as np

def remove_outliers(df):
    df_clean = df.groupby(['n', 'n_threads']).apply(lambda x: x[
        (x['time'] >= x['time'].quantile(0.05)) &
        (x['time'] <= x['time'].quantile(0.95))
    ]).reset_index(drop=True)
    return df_clean

def calculate_speedup(df):
    df_speedup = df.copy()
    for n in df['n'].unique():
        baseline = df[(df['n'] == n) & (df['n_threads'] == 1)]['time'].mean()
        df_speedup.loc[df_speedup['n'] == n, 'speedup'] = baseline / df_speedup.loc[df_speedup['n'] == n, 'time']
    return df_speedup

def analyze_speedup_and_time(df):
    # Remove outliers
    df_clean = remove_outliers(df)

    # Calculate speedup
    df_analysis = calculate_speedup(df_clean)

    # Aggregate results
    speedup_table = df_analysis.groupby(['n_threads', 'n'])['speedup'].mean().reset_index()
    time_table = df_analysis.groupby(['n_threads', 'n'])['time'].mean().reset_index()

    # Sort tables
    speedup_table = speedup_table.sort_values(['n', 'n_threads'])
    time_table = time_table.sort_values(['n', 'n_threads'])

    print("Speedup Table:")
    print(speedup_table.to_string(index=False))
    print("\nExecution Time Table:")
    print(time_table.to_string(index=False))
    
    return speedup_table, time_table


speedUp, time_table = analyze_speedup_and_time(df)

Speedup Table:
 n_threads      n   speedup
         1  20000  1.000273
         6  20000  4.953333
        11  20000  6.422095
        22  20000  6.429333
         1  40000  1.000003
         6  40000  5.062589
        11  40000  6.563328
        22  40000  6.654195
         1  80000  1.000001
         6  80000  5.105114
        11  80000  6.728793
        22  80000  6.764904
         1 120000  1.000001
         6 120000  5.090177
        11 120000  5.982493
        22 120000  6.724178
         1 140000  1.000007
         6 140000  4.911875
        11 140000  6.594455
        22 140000  6.730396
         1 160000  1.000029
         6 160000  5.054307
        11 160000  6.438948
        22 160000  6.717701
         1 180000  1.000079
         6 180000  5.139362
        11 180000  6.594127
        22 180000  6.787535
         1 200000  1.293394
         6 200000  8.329466
        11 200000 10.748376
        22 200000 10.727264

Execution Time Table:
 n_threads      n        time
        

In [36]:
time_pivot = time_table.pivot(index='n', columns='n_threads', values='time').reset_index()
speedup_pivot = speedUp.pivot(index='n', columns='n_threads', values='speedup').reset_index()

In [37]:
time_pivot

n_threads,n,1,6,11,22
0,20000,10.450350,2.109762,1.627345,1.625435
1,40000,42.817125,8.457839,6.527250,6.435369
2,80000,170.341250,33.367175,25.315375,25.180212
3,120000,383.146625,75.274987,75.400788,56.980787
4,140000,522.937000,106.625750,79.308437,77.701413
5,160000,683.530500,135.237375,106.159250,101.762500
6,180000,869.146750,169.117250,132.118000,128.058500
7,200000,1736.620000,208.492625,161.584375,161.901250


In [38]:
speedup_pivot

n_threads,n,1,6,11,22
0,20000,1.000273,4.953333,6.422095,6.429333
1,40000,1.000003,5.062589,6.563328,6.654195
2,80000,1.000001,5.105114,6.728793,6.764904
3,120000,1.000001,5.090177,5.982493,6.724178
4,140000,1.000007,4.911875,6.594455,6.730396
5,160000,1.000029,5.054307,6.438948,6.717701
6,180000,1.000079,5.139362,6.594127,6.787535
7,200000,1.293394,8.329466,10.748376,10.727264


In [39]:
hola = speedUp.groupby('n_threads')['speedup'].describe().drop(columns='count').iloc[1:]

In [42]:
hola['mean'].mean()

6.5522643722165315

In [30]:
time_table.groupby(['n_threads','n'])['time'].describe().drop(columns=['count','std'])

mean          min          25%          50%  \
n_threads n                                                            
1         20000     10.450350    10.450350    10.450350    10.450350   
          40000     42.817125    42.817125    42.817125    42.817125   
          80000    170.341250   170.341250   170.341250   170.341250   
          120000   383.146625   383.146625   383.146625   383.146625   
          140000   522.937000   522.937000   522.937000   522.937000   
          160000   683.530500   683.530500   683.530500   683.530500   
          180000   869.146750   869.146750   869.146750   869.146750   
          200000  1736.620000  1736.620000  1736.620000  1736.620000   
6         20000      2.109762     2.109762     2.109762     2.109762   
          40000      8.457839     8.457839     8.457839     8.457839   
          80000     33.367175    33.367175    33.367175    33.367175   
          120000    75.274987    75.274987    75.274987    75.274987   
          140000   106.625750   106.625750   106.625750   106.625750   
          160000   135.237375   135.237375   135.237375   135.237375   
          180000   169.117250   169.117250   169.117250   169.117250   
          200000   208.492625   208.492625   208.492625   208.492625   
11        20000      1.627345     1.627345     1.627345     1.627345   
          40000      6.527250     6.527250     6.527250     6.527250   
          80000     25.315375    25.315375    25.315375    25.315375   
          120000    75.400788    75.400788    75.400788    75.400788   
          140000    79.308437    79.308437    79.308437    79.308437   
          160000   106.159250   106.159250   106.159250   106.159250   
          180000   132.118000   132.118000   132.118000   132.118000   
          200000   161.584375   161.584375   161.584375   161.584375   
22        20000      1.625435     1.625435     1.625435     1.625435   
          40000      6.435369     6.435369     6.435369     6.435369   
          80000     25.180212    25.180212    25.180212    25.180212   
          120000    56.980787    56.980787    56.980787    56.980787   
          140000    77.701413    77.701413    77.701413    77.701413   
          160000   101.762500   101.762500   101.762500   101.762500   
          180000   128.058500   128.058500   128.058500   128.058500   
          200000   161.901250   161.901250   161.901250   161.901250   

                          75%          max  
n_threads n                                 
1         20000     10.450350    10.450350  
          40000     42.817125    42.817125  
          80000    170.341250   170.341250  
          120000   383.146625   383.146625  
          140000   522.937000   522.937000  
          160000   683.530500   683.530500  
          180000   869.146750   869.146750  
          200000  1736.620000  1736.620000  
6         20000      2.109762     2.109762  
          40000      8.457839     8.457839  
          80000     33.367175    33.367175  
          120000    75.274987    75.274987  
          140000   106.625750   106.625750  
          160000   135.237375   135.237375  
          180000   169.117250   169.117250  
          200000   208.492625   208.492625  
11        20000      1.627345     1.627345  
          40000      6.527250     6.527250  
          80000     25.315375    25.315375  
          120000    75.400788    75.400788  
          140000    79.308437    79.308437  
          160000   106.159250   106.159250  
          180000   132.118000   132.118000  
          200000   161.584375   161.584375  
22        20000      1.625435     1.625435  
          40000      6.435369     6.435369  
          80000     25.180212    25.180212  
          120000    56.980787    56.980787  
          140000    77.701413    77.701413  
          160000   101.762500   101.762500  
          180000   128.058500   128.058500  
          200000   161.901250   161.901250